Q1) What are the parameters of batchnorm? What information do you need to train the parameters?  

A1) $ \frac{X - \mu}{\sigma} * \gamma + \beta $, in which $\gamma$ and $\beta$ are optimized using training data (i.e. parameter) and $\mu$ and $\sigma$ attained from data.

Q2) Why do we use 'exponentially weighted moving average' (chain of linear interpolation) of training data optimizing two parameters? In other words, why can't we use one batch of training set's mean and variance in inference time?  
A2) When we get a totally different type of image at inference time, we can not fairly access/evaluate the parameters since attained mean/variance of training data are irrevant to inference.

In [1]:
!git clone https://github.com/fastai/course-v3/

Cloning into 'course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.10 MiB | 33.69 MiB/s, done.
Resolving deltas: 100% (3251/3251), done.
Checking out files: 100% (887/887), done.


In [4]:
%cd /content/course-v3/nbs/dl2/
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.nb_06 import *

/content/course-v3/nbs/dl2


In [6]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

- get data and transform those to regularized size
- dataset class, which enables slicing and length information
- databunch instance, which gives 1) dataloader 2) number of class
    1. dataloader
        - generate training data in a random order (when shuffle=True)
        - in case tensors don't have same length, pad first or last (e.g. pytorch [pad_packed_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_packed_sequence.html))
- callback functions
    - recorder which records learning rate and loss
    - avgstats which records total loss and batch count
    - cuda : convert to gpu before batch
    - unflatten image

- define convolution channels 


In [9]:
# ??AvgStats

In [11]:
x_train, y_train, x_valid, y_valid = get_data()
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)
mnist_view = view_tfm(1, 28, 28)
cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        # CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]
nfs = [8, 16, 32, 64, 64]
learn, run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

%time run.fit(2, learn)

train: [0.97408328125, tensor(0.6995)]
valid: [0.167479296875, tensor(0.9488)]
train: [0.17027271484375, tensor(0.9479)]
valid: [0.11941851806640626, tensor(0.9643)]
CPU times: user 23.6 s, sys: 674 ms, total: 24.3 s
Wall time: 12.6 s
